In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from scipy import stats

## Longer time series data

### Combining SDL + OPT data

In [2]:
df_sdl_2016_2021 = pd.read_csv("Raw_Files/sdl_gross_add_3yrs_q321.csv")
df_sdl_2016_2021.head()

,period,DMA,gross_add,gross_add_revenue
0,2015-12-27,"ABILENE, TX",1.0,2500.00
1,2015-12-27,"HOUSTON, TX",1.0,64.95
2,2015-12-27,"MONROE-ELDORADO, LA",1.0,45.36
3,2015-12-27,"PARKERSBERG, WV",1.0,89.95
4,2016-01-03,"ABILENE, TX",23.0,2069.03


In [3]:
df_sdl_2016_2021['DMA'].unique()

array(['ABILENE, TX', 'HOUSTON, TX', 'MONROE-ELDORADO, LA',
       'PARKERSBERG, WV', 'ALEXANDRIA, LA', 'AMARILLO, TX', 'AUSTIN, TX',
       'BLUEFIELD-BECKLEY, WV', 'CHARLESTON-HUNTINGTON, WV',
       'CLARKSBURG-WESTON, WV', 'COLUMBIA-JEFFERSON CITY, MO',
       'DALLAS, TX', 'EUREKA, CA', 'GREENVILLE-NEW BERN, NC',
       'GREENWOOD-GREENVILLE, MS', 'JONESBORO, AR',
       'JOPLIN, MO - PITTSBURG, KS', 'KANSAS CITY, MO',
       'LAKE CHARLES, LA', 'LAS VEGAS, NV', 'LITTLE ROCK, AR',
       'LOS ANGELES, CA', 'LUBBOCK, TX', 'MEMPHIS, TN',
       'ODESSA-MIDLAND, TX', 'OKLAHOMA CITY, OK', 'PALM SPRINGS, CA',
       'PHOENIX, AZ', 'RALEIGH-DURHAM, NC', 'RENO, NV', 'SACRAMENTO, CA',
       'SAN ANGELO, TX', 'SHREVEPORT, LA', 'SPOKANE, WA',
       'SPRINGFIELD, MO', 'ST JOSEPH, MO', 'TULSA, OK',
       'TYLER-LONGVIEW, TX', 'VICTORIA, TX', 'WACO-TEMPLE, TX',
       'WICHITA FALLS, TX - LAWTON, OK', 'LAFAYETTE, LA',
       'SHERMAN-ADA, OK', 'MONTEREY-SALINAS, CA', 'CLEVELAND, OH',
      

In [4]:
df_sdl_2016_2021['DMA'].nunique()

49

In [5]:
df_sdl_2016_2021[df_sdl_2016_2021['period'] == '2019-12-01']['DMA'].unique()

array(['ABILENE, TX', 'ALEXANDRIA, LA', 'AMARILLO, TX', 'AUSTIN, TX',
       'BLUEFIELD-BECKLEY, WV', 'CHARLESTON-HUNTINGTON, WV',
       'CLARKSBURG-WESTON, WV', 'COLUMBIA-JEFFERSON CITY, MO',
       'DALLAS, TX', 'EUREKA, CA', 'GREENVILLE-NEW BERN, NC',
       'GREENWOOD-GREENVILLE, MS', 'HOUSTON, TX', 'JONESBORO, AR',
       'KANSAS CITY, MO', 'LAKE CHARLES, LA', 'LITTLE ROCK, AR',
       'LUBBOCK, TX', 'MONROE-ELDORADO, LA', 'ODESSA-MIDLAND, TX',
       'OKLAHOMA CITY, OK', 'PALM SPRINGS, CA', 'PARKERSBERG, WV',
       'PHOENIX, AZ', 'RALEIGH-DURHAM, NC', 'RENO, NV', 'SACRAMENTO, CA',
       'SAN ANGELO, TX', 'SHREVEPORT, LA', 'SPRINGFIELD, MO',
       'ST JOSEPH, MO', 'TULSA, OK', 'TYLER-LONGVIEW, TX', 'VICTORIA, TX',
       'WACO-TEMPLE, TX', 'WICHITA FALLS, TX - LAWTON, OK'], dtype=object)

In [6]:
df_opt_2016_2021 = pd.read_csv("Raw_Files/opt_gross_add_3yrs_q321.csv")
df_opt_2016_2021.head()

,period,gross_add,gross_add_revenue
0,2015-12-27,48.0,6055.72
1,2016-01-03,960.0,101760.85
2,2016-01-10,943.0,98930.51
3,2016-01-17,815.0,83757.10
4,2016-01-24,894.0,100049.23


In [7]:
df_opt_2016_2021['DMA'] = 'NEW YORK, NY'
df_opt_2016_2021.head()

,period,gross_add,gross_add_revenue,DMA
0,2015-12-27,48.0,6055.72,"NEW YORK, NY"
1,2016-01-03,960.0,101760.85,"NEW YORK, NY"
2,2016-01-10,943.0,98930.51,"NEW YORK, NY"
3,2016-01-17,815.0,83757.10,"NEW YORK, NY"
4,2016-01-24,894.0,100049.23,"NEW YORK, NY"


In [8]:
df_sdl_opt_2016_2021 = pd.concat([df_sdl_2016_2021,df_opt_2016_2021], axis = 0)
# df_sdl_opt_2016_2021.to_csv('/opt/notebooks/MMM_B2B/Data/B2B_GA_2016_2019.csv')

In [9]:
df_sdl_opt_2016_2021

,period,DMA,gross_add,gross_add_revenue
0,2015-12-27,"ABILENE, TX",1.0,2500.00
1,2015-12-27,"HOUSTON, TX",1.0,64.95
2,2015-12-27,"MONROE-ELDORADO, LA",1.0,45.36
3,2015-12-27,"PARKERSBERG, WV",1.0,89.95
4,2016-01-03,"ABILENE, TX",23.0,2069.03
...,...,...,...,...
269,2021-02-21,"NEW YORK, NY",1002.0,143013.30
270,2021-02-28,"NEW YORK, NY",917.0,121802.20
271,2021-03-07,"NEW YORK, NY",856.0,121021.04
272,2021-03-14,"NEW YORK, NY",875.0,125020.15


In [10]:
# df_sdl_opt_2016_2019.to_csv('/opt/notebooks/MMM_B2B/Data/B2B_GA_2016_2019.csv')

### getting all 51 DMA names for all months

SAN ANTONIO, TX has 0 GA. Hence that DMA is not present in data. It will be included in the All Other Suddenlink DMA total in section 1.3

In [11]:
# taking act_vol_dvr as column - no specific reason
df_pivot = pd.pivot_table(df_sdl_opt_2016_2021, values='gross_add', index=['period'], columns=['DMA'], aggfunc=np.sum)
df_pivot.reset_index(inplace = True)
df_pivot.head()

DMA,period,"ABILENE, TX","ALEXANDRIA, LA","AMARILLO, TX","AUSTIN, TX","BLUEFIELD-BECKLEY, WV","CHARLESTON-HUNTINGTON, WV","CLARKSBURG-WESTON, WV","CLEVELAND, OH","COLUMBIA-JEFFERSON CITY, MO",...,"SHREVEPORT, LA","SPOKANE, WA","SPRINGFIELD, MO","ST JOSEPH, MO","TULSA, OK","TYLER-LONGVIEW, TX","VICTORIA, TX","WACO-TEMPLE, TX","WICHITA FALLS, TX - LAWTON, OK","WICHITA-HUTCHINSON, KS"
0,2015-12-27,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-03,23.0,11.0,18.0,16.0,14.0,22.0,2.0,NaN,1.0,...,17.0,1.0,14.0,10.0,6.0,51.0,16.0,13.0,1.0,NaN
2,2016-01-10,18.0,15.0,31.0,12.0,13.0,22.0,2.0,NaN,2.0,...,13.0,NaN,17.0,4.0,6.0,47.0,9.0,17.0,1.0,NaN
3,2016-01-17,15.0,11.0,23.0,13.0,7.0,26.0,3.0,NaN,NaN,...,11.0,NaN,17.0,5.0,6.0,41.0,7.0,14.0,2.0,NaN
4,2016-01-24,12.0,13.0,29.0,9.0,6.0,41.0,1.0,NaN,NaN,...,15.0,4.0,17.0,3.0,8.0,39.0,6.0,19.0,3.0,NaN


In [12]:
df_pivot.fillna(0,inplace = True)
df_pivot.reset_index(inplace = True)
df_pivot.head()

DMA,index,period,"ABILENE, TX","ALEXANDRIA, LA","AMARILLO, TX","AUSTIN, TX","BLUEFIELD-BECKLEY, WV","CHARLESTON-HUNTINGTON, WV","CLARKSBURG-WESTON, WV","CLEVELAND, OH",...,"SHREVEPORT, LA","SPOKANE, WA","SPRINGFIELD, MO","ST JOSEPH, MO","TULSA, OK","TYLER-LONGVIEW, TX","VICTORIA, TX","WACO-TEMPLE, TX","WICHITA FALLS, TX - LAWTON, OK","WICHITA-HUTCHINSON, KS"
0,0,2015-12-27,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2016-01-03,23.0,11.0,18.0,16.0,14.0,22.0,2.0,0.0,...,17.0,1.0,14.0,10.0,6.0,51.0,16.0,13.0,1.0,0.0
2,2,2016-01-10,18.0,15.0,31.0,12.0,13.0,22.0,2.0,0.0,...,13.0,0.0,17.0,4.0,6.0,47.0,9.0,17.0,1.0,0.0
3,3,2016-01-17,15.0,11.0,23.0,13.0,7.0,26.0,3.0,0.0,...,11.0,0.0,17.0,5.0,6.0,41.0,7.0,14.0,2.0,0.0
4,4,2016-01-24,12.0,13.0,29.0,9.0,6.0,41.0,1.0,0.0,...,15.0,4.0,17.0,3.0,8.0,39.0,6.0,19.0,3.0,0.0


In [13]:
df_pivot.columns

Index(['index', 'period', 'ABILENE, TX', 'ALEXANDRIA, LA', 'AMARILLO, TX',
       'AUSTIN, TX', 'BLUEFIELD-BECKLEY, WV', 'CHARLESTON-HUNTINGTON, WV',
       'CLARKSBURG-WESTON, WV', 'CLEVELAND, OH', 'COLUMBIA-JEFFERSON CITY, MO',
       'COLUMBUS, OH', 'DALLAS, TX', 'EUREKA, CA', 'FRESNO-VISALIA, CA',
       'FT SMITH, AR', 'GREENVILLE-NEW BERN, NC', 'GREENWOOD-GREENVILLE, MS',
       'HOUSTON, TX', 'JONESBORO, AR', 'JOPLIN, MO - PITTSBURG, KS',
       'KANSAS CITY, MO', 'LAFAYETTE, LA', 'LAKE CHARLES, LA', 'LAS VEGAS, NV',
       'LITTLE ROCK, AR', 'LOS ANGELES, CA', 'LUBBOCK, TX', 'MEMPHIS, TN',
       'MONROE-ELDORADO, LA', 'MONTEREY-SALINAS, CA', 'NEW YORK, NY',
       'ODESSA-MIDLAND, TX', 'OKLAHOMA CITY, OK', 'PALM SPRINGS, CA',
       'PARKERSBERG, WV', 'PHOENIX, AZ', 'RALEIGH-DURHAM, NC', 'RENO, NV',
       'SACRAMENTO, CA', 'SAN ANGELO, TX', 'SHERMAN-ADA, OK', 'SHREVEPORT, LA',
       'SPOKANE, WA', 'SPRINGFIELD, MO', 'ST JOSEPH, MO', 'TULSA, OK',
       'TYLER-LONGVIEW, TX', 

In [14]:
df_pivot.drop(columns = ['index'], axis = 1, inplace = True)
df_pivot.head()

DMA,period,"ABILENE, TX","ALEXANDRIA, LA","AMARILLO, TX","AUSTIN, TX","BLUEFIELD-BECKLEY, WV","CHARLESTON-HUNTINGTON, WV","CLARKSBURG-WESTON, WV","CLEVELAND, OH","COLUMBIA-JEFFERSON CITY, MO",...,"SHREVEPORT, LA","SPOKANE, WA","SPRINGFIELD, MO","ST JOSEPH, MO","TULSA, OK","TYLER-LONGVIEW, TX","VICTORIA, TX","WACO-TEMPLE, TX","WICHITA FALLS, TX - LAWTON, OK","WICHITA-HUTCHINSON, KS"
0,2015-12-27,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-01-03,23.0,11.0,18.0,16.0,14.0,22.0,2.0,0.0,1.0,...,17.0,1.0,14.0,10.0,6.0,51.0,16.0,13.0,1.0,0.0
2,2016-01-10,18.0,15.0,31.0,12.0,13.0,22.0,2.0,0.0,2.0,...,13.0,0.0,17.0,4.0,6.0,47.0,9.0,17.0,1.0,0.0
3,2016-01-17,15.0,11.0,23.0,13.0,7.0,26.0,3.0,0.0,0.0,...,11.0,0.0,17.0,5.0,6.0,41.0,7.0,14.0,2.0,0.0
4,2016-01-24,12.0,13.0,29.0,9.0,6.0,41.0,1.0,0.0,0.0,...,15.0,4.0,17.0,3.0,8.0,39.0,6.0,19.0,3.0,0.0


In [15]:
df_melt = pd.melt(df_pivot, id_vars=['period']).sort_values(by=['period'])

In [16]:
df_melt

,period,DMA,value
0,2015-12-27,"ABILENE, TX",1.0
9590,2015-12-27,"RALEIGH-DURHAM, NC",0.0
1370,2015-12-27,"CHARLESTON-HUNTINGTON, WV",0.0
2192,2015-12-27,"COLUMBIA-JEFFERSON CITY, MO",0.0
6302,2015-12-27,"LITTLE ROCK, AR",0.0
...,...,...,...
7945,2021-03-21,"MONTEREY-SALINAS, CA",1.0
7671,2021-03-21,"MONROE-ELDORADO, LA",2.0
7397,2021-03-21,"MEMPHIS, TN",0.0
6575,2021-03-21,"LITTLE ROCK, AR",12.0


In [17]:
df_melt.groupby('period').agg({'DMA':'count'})['DMA'].unique()

array([50], dtype=int64)

In [18]:
df_melt.shape

(13700, 3)

In [19]:
df_melt.groupby(['period']).agg({'DMA':'count'})

,DMA
period,
2015-12-27,50
2016-01-03,50
2016-01-10,50
2016-01-17,50
2016-01-24,50
...,...
2021-02-21,50
2021-02-28,50
2021-03-07,50


### Reducing DMAs from 51 to 30

In [20]:
df_melt.head()

,period,DMA,value
0,2015-12-27,"ABILENE, TX",1.0
9590,2015-12-27,"RALEIGH-DURHAM, NC",0.0
1370,2015-12-27,"CHARLESTON-HUNTINGTON, WV",0.0
2192,2015-12-27,"COLUMBIA-JEFFERSON CITY, MO",0.0
6302,2015-12-27,"LITTLE ROCK, AR",0.0


In [21]:
DMA_to_merge = ['GREENWOOD-GREENVILLE, MS','SACRAMENTO, CA','CLARKSBURG-WESTON, WV','KANSAS CITY, MO','PALM SPRINGS, CA',
                'WICHITA FALLS, TX - LAWTON, OK','JOPLIN, MO - PITTSBURG, KS','RENO, NV','LOS ANGELES, CA','MEMPHIS, TN',
                'LAFAYETTE, LA','LAS VEGAS, NV','CLEVELAND, OH','COLUMBIA-JEFFERSON CITY, MO','FT SMITH, AR','MONTEREY-SALINAS, CA',
                'SPOKANE, WA','COLUMBUS, OH','SHERMAN-ADA, OK','SAN ANTONIO, TX','WICHITA-HUTCHINSON, KS','FRESNO-VISALIA, CA']
len(DMA_to_merge)

22

In [22]:
df_to_reduce = df_melt[df_melt['DMA'].isin(DMA_to_merge)]
df_to_remain = df_melt[~df_melt['DMA'].isin(DMA_to_merge)]

print(df_to_reduce.shape)
print(df_to_remain.shape)

(5754, 3)
(7946, 3)


In [23]:
df_to_reduce.columns

Index(['period', 'DMA', 'value'], dtype='object')

In [24]:
cols_for_addition = ['period', 'value']

df_to_sum = df_to_reduce[cols_for_addition]
dfsum = df_to_sum.groupby(['period']).sum().reset_index()

In [25]:
dfsum

,period,value
0,2015-12-27,0.0
1,2016-01-03,36.0
2,2016-01-10,32.0
3,2016-01-17,27.0
4,2016-01-24,43.0
...,...,...
269,2021-02-21,19.0
270,2021-02-28,33.0
271,2021-03-07,25.0
272,2021-03-14,30.0


In [26]:
dfsum['DMA'] = 'ALL OTHER SUDDENLINK DMA TOTAL'
dfsum.shape

(274, 3)

In [27]:
dfsum.head()

,period,value,DMA
0,2015-12-27,0.0,ALL OTHER SUDDENLINK DMA TOTAL
1,2016-01-03,36.0,ALL OTHER SUDDENLINK DMA TOTAL
2,2016-01-10,32.0,ALL OTHER SUDDENLINK DMA TOTAL
3,2016-01-17,27.0,ALL OTHER SUDDENLINK DMA TOTAL
4,2016-01-24,43.0,ALL OTHER SUDDENLINK DMA TOTAL


In [28]:
df_full = pd.concat([df_to_remain, dfsum])
df_full.shape

(8220, 3)

In [29]:
df_full.head()

,period,DMA,value
0,2015-12-27,"ABILENE, TX",1.0
9590,2015-12-27,"RALEIGH-DURHAM, NC",0.0
1370,2015-12-27,"CHARLESTON-HUNTINGTON, WV",0.0
6302,2015-12-27,"LITTLE ROCK, AR",0.0
4384,2015-12-27,"HOUSTON, TX",1.0


In [30]:
df_full.rename(columns = {'value':'gross_add','DMA':'dma'}, inplace = True)
df_full.head()

,period,dma,gross_add
0,2015-12-27,"ABILENE, TX",1.0
9590,2015-12-27,"RALEIGH-DURHAM, NC",0.0
1370,2015-12-27,"CHARLESTON-HUNTINGTON, WV",0.0
6302,2015-12-27,"LITTLE ROCK, AR",0.0
4384,2015-12-27,"HOUSTON, TX",1.0


In [31]:
df_full[['period','dma','gross_add']].to_csv('Data/B2B_GA_2016_2021_30_DMAs.csv',index = False)